In [1]:
from PIL import Image
import os
import cv2
import shutil
import numpy as np

## Rename imágenes originales

In [1]:
def rename_img(carpeta_origen, carpeta_destino):
    # Crea la carpeta de destino si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    # Lista de nombres de archivos en la carpeta de origen
    archivos = sorted(os.listdir(carpeta_origen))

    # Contador para el nuevo nombre
    contador = 0

    # Iterar sobre los archivos y renombrarlos, de forma consecutiva
    for archivo in archivos:
        if archivo.endswith((".jpg", ".png")):
            nuevo_nombre = f"{contador:03d}.png"
            ruta_origen = os.path.join(carpeta_origen, archivo)
            ruta_destino = os.path.join(carpeta_destino, nuevo_nombre)
            shutil.copy(ruta_origen, ruta_destino)
            contador += 1

    print("Imágenes renombradas y copiadas exitosamente.")

## Redimensión y mapeo

In [2]:
def process_images(origen_images, origen_label, idx_train, idx_test, dest_train_images, dest_train_label, dest_test_images, dest_test_label, map_pixels_intermediate, map_pixels_final):
    
    # Creamos las rutas si no existen
    for carpeta_destino in [dest_train_images, dest_train_label, dest_test_images, dest_test_label]:
        if not os.path.exists(carpeta_destino):
            os.makedirs(carpeta_destino)

    # Si el indice esta en train, va para la carpeta train sino, para test
    for i in range(400):
        if i in idx_train:
            dest_images = dest_train_images
            dest_labels = dest_train_label
        elif i in idx_test:
            dest_images = dest_test_images
            dest_labels = dest_test_label
        else:
            continue

        # Redimensionar y guardar imagen original
        img = Image.open(os.path.join(origen_images, f"{i:03d}.png"))
        img = img.resize((256,256), Image.NEAREST)
        img.save(os.path.join(dest_images,f"{i:03d}.png"))

        mask = Image.open(os.path.join(origen_label,f"{i:03d}.png"))
        mask_np = np.array(mask)

        # Mapeo
        for k,v in map_pixels_intermediate.items():
            mask_np[mask_np==k] = v

        for k,v in map_pixels_final.items():
            mask_np[mask_np==k] = v

        mask_remap = Image.fromarray(mask_np)
        mask_remap = mask_remap.resize((256,256), Image.NEAREST)
        mask_remap.save(os.path.join(dest_labels,f"{i:03d}.png"))

## Data Augmentation 

In [3]:
def transform_images(image_dir, is_mask):
    # Lista de ángulos para la rotación
    angles = [45, 90, 135, 180, 225, 270, 315]

    # Recorre todas las imágenes en el directorio dado
    for filename in os.listdir(image_dir):
        if filename.endswith(".png"):
            
            number = filename.split(".")[0]
            
            # Carga imagen
            img = cv2.imread(os.path.join(image_dir, filename))

            # Crea y guarda imagen espejo
            img_mirror = cv2.flip(img, 1)
            cv2.imwrite(os.path.join(image_dir, f"{number}_mirror.png"), img_mirror)

            # Crea y guarda imágenes rotadas
            for angle in angles:
                
                # Si es mascara aplicamos INERT_NEAREST para no variar los valores de los pixeles (pertenecen a clases) y elige el pixel mas cercano
                # Si no lo es aplicamos INTER_LINEAR hace que la redimension sea mas suave y con detalle
                if is_mask:
                    interpolation = cv2.INTER_NEAREST
                else:
                    interpolation = cv2.INTER_LINEAR
                
                M = cv2.getRotationMatrix2D((int(img.shape[1]/2), int(img.shape[0]/2)), angle, 1)
                img_rot = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), flags=interpolation, borderMode=cv2.BORDER_REFLECT)
                cv2.imwrite(os.path.join(image_dir, f"{number}_rot{angle}.png"), img_rot)

                M_mirror = cv2.getRotationMatrix2D((int(img_mirror.shape[1]/2), int(img_mirror.shape[0]/2)), angle, 1)
                img_mirror_rot = cv2.warpAffine(img_mirror, M_mirror, (img_mirror.shape[1], img_mirror.shape[0]), flags=interpolation, borderMode=cv2.BORDER_REFLECT)
                cv2.imwrite(os.path.join(image_dir, f"{number}_mirror_rot{angle}.png"), img_mirror_rot)

                # Ajustar brillo y contraste, si es una mascara no aplicamos cambios, para no variar los valores de los pixeles
                if is_mask == False: 
                    img_bc = cv2.convertScaleAbs(img, alpha=1.3, beta=10)
                elif is_mask == True:
                    img_bc = img
                
                cv2.imwrite(os.path.join(image_dir, f"{number}_bc.png"), img_bc)

                # Ajustar brillo y contraste para la imagen espejo, igual que antes si es una mascara no aplicamos transformacion
                if is_mask == False:
                    img_mirror_bc = cv2.convertScaleAbs(img_mirror, alpha=1.3, beta=10)
                elif is_mask == True:
                    img_mirror_bc = img_mirror
                        
                cv2.imwrite(os.path.join(image_dir, f"{number}_mirror_bc.png"), img_mirror_bc)

                # Crear y guardar imágenes rotadas para la imagen con brillo y contraste ajustados
                for angle in angles:
                    
                    # Rotamos las imagenes con el brillo y contraste ajustados
                    M = cv2.getRotationMatrix2D((int(img_bc.shape[1]/2), int(img_bc.shape[0]/2)), angle, 1)
                    
                    img_bc_rot = cv2.warpAffine(img_bc, M, (img_bc.shape[1], img_bc.shape[0]),flags=interpolation, borderMode=cv2.BORDER_REFLECT)
                    
                    cv2.imwrite(os.path.join(image_dir,f"{number}_bc_rot{angle}.png"),img_bc_rot)

                    # Rotamos las imagenes espejo con el brillo y contraste ajustados
                    M_mirror_bc = cv2.getRotationMatrix2D((int(img_mirror_bc.shape[1] / 2),int(img_mirror_bc.shape[0] / 2)),angle,1)
                    
                    img_mirror_bc_rot = cv2.warpAffine(img_mirror_bc,M_mirror_bc,(img_mirror_bc.shape[1],img_mirror_bc.shape[0]),flags=interpolation,borderMode=cv2.BORDER_REFLECT)
                    
                    cv2.imwrite(os.path.join(image_dir,f"{number}_mirror_bc_rot{angle}.png"),img_mirror_bc_rot)

## Comprobamos concordancia de nombres despues de las transformaciones

In [4]:
# Comprobamos correlacion de archivos imagen - mascara
def compare_mask_img(dir1, dir2):
    
    # Obtenemos ficheros
    archivos_1 = set(os.listdir(dir1))
    archivos_2 = set(os.listdir(dir2))
    
    return archivos_1 == archivos_2